In [1]:
import  pickle
import numpy as np
circuit_paths = [ 'random_transpiled_circuit_3400.pkl', 'circuit20_20.pkl', 'alg_transpiled_circuit_13.pkl',]

In [2]:
single_average_error = {0: 1-2.266e-4, #rb
 1: 1- 1.699e-4,
 2: 1- 6.082e-4,
 3: 1- 1.686e-4,
 4: 1- 3.998e-4,
}
couple_average_error = {(0, 1): 1- 0.00509, # rb
                        (1, 0): 1- 0.00509,
                        (1, 2): 1- 0.01109,
                        (2, 1): 1- 0.01109,
                        (2, 3): 1- 0.0066,
                        (3, 2): 1- 0.0066,
                        (3, 4): 1- 0.00586,
                        (4, 3): 1- 0.00586,
}

In [3]:
from upstream.randomwalk_model import extract_device


def get_rb_fidelity(circuit_info):
    fidelity = 1
    for gate in circuit_info['gates']:
        device = extract_device(gate)
        if isinstance(device,tuple):
            if 'map' in circuit_info:
                device = (circuit_info['map'][device[0]],circuit_info['map'][device[1]])
            fidelity = fidelity * couple_average_error[device]
        else:
            if 'map' in circuit_info:
                device = circuit_info['map'][device]
            fidelity = fidelity * single_average_error[device]
    return fidelity #* np.product((measure0_fidelity + measure1_fidelity) / 2)

In [4]:

circuits = []
for circuit_path in circuit_paths:
    with open(circuit_path,'rb') as f:
        circuits += pickle.load(f)
circuits = circuits[1:]

In [5]:
with open('counts.pkl','rb') as f:
    counts = pickle.load(f)

In [7]:
from circuit import qiskit_to_layered_circuits
dataset = []
reals = []
for cir, count in zip(circuits[:3399] ,counts[:3399]):
    if '00000' not in count:
        continue
    ground_truth_fidelity = count['00000'] / 5000
    if ground_truth_fidelity < 0.2:
        # print('bad')
        continue

    reals.append(ground_truth_fidelity)
    circuit_info = qiskit_to_layered_circuits(cir)
    circuit_info['ground_truth_fidelity'] = ground_truth_fidelity
    dataset.append(circuit_info)

In [8]:
import random
random.shuffle(dataset)

In [10]:
train_dataset, test_dataset = dataset[:-400], dataset[-400:]
len(train_dataset), len(test_dataset), len(circuits[3399:])

(2192, 400, 530)

In [11]:
test_predicts, test_reals = [] , []
for circuit_info in test_dataset:

    predict = get_rb_fidelity(circuit_info)
    circuit_info['predict'] = predict

    test_reals.append(circuit_info['ground_truth_fidelity'])
    test_predicts.append(predict) 
    print(circuit_info['ground_truth_fidelity'], predict)
print(np.abs(np.array(test_reals)-np.array(test_predicts)).mean())

0.4944 0.6678509802978404
0.925 0.9469658390684612
0.4244 0.7742728324460303
0.423 0.8040678134200461
0.3972 0.7008356134919265
0.3106 0.7003518966441653
0.6802 0.809973250837274
0.24 0.7156783094216749
0.2508 0.6779666623011199
0.394 0.7686097044610001
0.2362 0.6272294031379583
0.3966 0.8398657928530041
0.5918 0.8181351410346389
0.6248 0.8491258694868975
0.662 0.7954219697669029
0.8756 0.9171675389561613
0.2332 0.6241670995670585
0.685 0.8509227107163394
0.205 0.7212643522668648
0.5414 0.7325671445715615
0.6906 0.866952434125973
0.7618 0.8766290480317896
0.2458 0.5874668678487458
0.2434 0.6732048771345331
0.226 0.6645265822505718
0.3048 0.6393488072620735
0.2608 0.7633718014838649
0.2364 0.5792688948687911
0.546 0.7528733921234787
0.3412 0.7689538203100189
0.7484 0.8530916763931116
0.8222 0.880372957053582
0.8348 0.9113391349609912
0.4508 0.8068998309576463
0.351 0.6857466225947907
0.4804 0.7934667490866469
0.4026 0.7079057603915371
0.2724 0.6700307655482695
0.5344 0.7994763190690767


In [12]:
rb = np.abs(np.array(test_reals)-np.array(test_predicts)).tolist()
pickle.dump(rb ,open('rb.pkl','wb'))

[0.1734509802978404, 0.021965839068461168, 0.3498728324460303, 0.3810678134200461, 0.30363561349192647, 0.3897518966441653, 0.12977325083727398, 0.4756783094216749, 0.4271666623011199, 0.37460970446100006, 0.3910294031379583, 0.4432657928530041, 0.22633514103463892, 0.22432586948689748, 0.1334219697669029, 0.041567538956161276, 0.39096709956705855, 0.16592271071633935, 0.5162643522668648, 0.19116714457156148, 0.176352434125973, 0.11482904803178962, 0.3416668678487458, 0.42980487713453314, 0.43852658225057184, 0.33454880726207353, 0.5025718014838649, 0.3428688948687911, 0.20687339212347866, 0.4277538203100189, 0.10469167639311161, 0.05817295705358194, 0.07653913496099118, 0.3560998309576463, 0.3347466225947907, 0.31306674908664694, 0.30530576039153706, 0.3976307655482695, 0.2650763190690767, 0.34532819473078163, 0.34302114926721095, 0.4313155979550678, 0.1321957415795787, 0.3712264693715371, 0.16806669875706748, 0.08025014547097231, 0.3566841408326404, 0.05106354246741773, 0.32769811517